In [28]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("data.csv")
df.head()

,Unnamed: 0,company_name,model_name,price,model_year,location,mileage,engine_type,engine_capacity,color,assembly,body_type,transmission_type,registration_status
0,0,Toyota,Vitz,2385000,2017,Islamabad,9869,Petrol,1000,Silver,Imported,Hatchback,Automatic,Un-Registered
1,1,Toyota,Corolla,111000,2019,KPK,11111,Petrol,1300,White,Local,Sedan,Automatic,Registered
2,2,Suzuki,Alto,1530000,2019,KPK,17500,Petrol,660,White,Local,Hatchback,Automatic,Un-Registered
3,3,Suzuki,Alto,1650000,2019,Punjab,9600,Petrol,660,White,Local,Hatchback,Manual,Registered
4,4,Toyota,Corolla,1435000,2010,Islamabad,120000,Petrol,1300,Black,Local,Sedan,Manual,Registered


In [18]:
df = df.drop("Unnamed: 0", axis = 1)
df.head()

,company_name,model_name,price,model_year,location,mileage,engine_type,engine_capacity,color,assembly,body_type,transmission_type,registration_status
0,Toyota,Vitz,2385000,2017,Islamabad,9869,Petrol,1000,Silver,Imported,Hatchback,Automatic,Un-Registered
1,Toyota,Corolla,111000,2019,KPK,11111,Petrol,1300,White,Local,Sedan,Automatic,Registered
2,Suzuki,Alto,1530000,2019,KPK,17500,Petrol,660,White,Local,Hatchback,Automatic,Un-Registered
3,Suzuki,Alto,1650000,2019,Punjab,9600,Petrol,660,White,Local,Hatchback,Manual,Registered
4,Toyota,Corolla,1435000,2010,Islamabad,120000,Petrol,1300,Black,Local,Sedan,Manual,Registered


In [19]:
df.isnull().sum()

company_name           0
model_name             0
price                  0
model_year             0
location               0
mileage                0
engine_type            0
engine_capacity        0
color                  0
assembly               0
body_type              0
transmission_type      0
registration_status    0
dtype: int64

In [20]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cat_columns = ["company_name", "model_name", "model_year", "location", "engine_type", "color", "assembly", "body_type", "transmission_type", "registration_status"]
for column in cat_columns:
    df[column + "encoded"] = encoder.fit_transform(df[column])
df.drop(columns = cat_columns, inplace = True)
df.head()

,price,mileage,engine_capacity,company_nameencoded,model_nameencoded,model_yearencoded,locationencoded,engine_typeencoded,colorencoded,assemblyencoded,body_typeencoded,transmission_typeencoded,registration_statusencoded
0,2385000,9869,1000,28,181,27,1,2,18,0,1,0,1
1,111000,11111,1300,28,53,29,2,2,21,1,4,0,0
2,1530000,17500,660,27,14,29,2,2,21,1,1,0,1
3,1650000,9600,660,27,14,29,4,2,21,1,1,1,0
4,1435000,120000,1300,28,53,20,1,2,2,1,4,1,0


In [22]:
from sklearn.preprocessing import StandardScaler
columns_to_scale = ["mileage", "engine_capacity"]
df_scaled = df.copy()
df_scaled[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
df_scaled.head()

,price,mileage,engine_capacity,company_nameencoded,model_nameencoded,model_yearencoded,locationencoded,engine_typeencoded,colorencoded,assemblyencoded,body_typeencoded,transmission_typeencoded,registration_statusencoded
0,2385000,-1.273975,-0.509393,28,181,27,1,2,18,0,1,0,1
1,111000,-1.254464,-0.021337,28,53,29,2,2,21,1,4,0,0
2,1530000,-1.154097,-1.062522,27,14,29,2,2,21,1,1,0,1
3,1650000,-1.278201,-1.062522,27,14,29,4,2,21,1,1,1,0
4,1435000,0.456122,-0.021337,28,53,20,1,2,2,1,4,1,0


In [53]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.linear_model import Lasso, Ridge
import numpy as np

# Ridge Regression
X = df_scaled.drop("price", axis=1)
y = df_scaled["price"]
kf = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {"alpha": np.arange(0.001, 1, 10), "solver": ["sag", "saga"]}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
ridge = Ridge()
random_search_ridge = RandomizedSearchCV(estimator=ridge, param_distributions=param_grid, random_state=42, cv=kf, n_iter = 2)
random_search_ridge.fit(X_train, y_train)
print(random_search_ridge.best_params_, random_search_ridge.best_score_)
print(random_search_ridge.score(X_test, y_test))

# Lasso Regression
param_grid = {"alpha": np.arange(0.001, 1, 10)}
lasso = Lasso()
random_search_lasso = RandomizedSearchCV(estimator=lasso, param_distributions=param_grid, random_state=42, cv=kf, n_jobs=-1)
random_search_lasso.fit(X_train, y_train)
print(random_search_lasso.best_params_, random_search_lasso.best_score_)
print(random_search_lasso.score(X_test, y_test))


{'solver': 'saga', 'alpha': 0.001} 0.5933742528289103
0.5687254012904941


C:\ProgramData\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'alpha': 0.001} 0.5933712363624535
0.5687187169909403


In [54]:
#Ridge model performed slightly better and will be used for predictions
